## Metaprogramming

In [1]:
## One of the most important mantras of software development is dont repeat yourself. That is nay time you are faced with a problem of creating highly repetitive code it often pays to look for a more elegant solution. In a nutshell metaprogramming is about creating functions and classes whose main goal is to manipulate code. the main features for this include decorators class decorators and metaclasses. However a variety of other useful topics including signature objects execution of code with exec() and inspecting the internals of funcitons and lcasses enter the picture. tThe main purpose of this chapter is to explore varous metaprogramming techniques and to give examples of how they can be used to customize the behavior of Python to your own whims.




## Putting a Wrapper Around a Function

In [6]:
#First create a wrapper using @wraps

import time
from functools import wraps

def timethis(func):
    ''' 
    Decorator that reports the execution time
    '''

    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

In [7]:
#Here is an exaple of using the decorator

@timethis
def countdown(n):
    while n>0:
        n -= 1

countdown(1000000)


countdown 0.0809779167175293


In [8]:
#decorator is a function that accepts a funciton as input and returns a new functon as ooutput. Whenever you write a code like this

@timethis
def countdown(n):
    pass

#its the same as if you had performed these separate steps:
def countdown(n):
    pass

countdown = timethis(countdown)

In [36]:
class A:
    @staticmethod
    def method(cls):
        cls.myvar = 1
        return cls.myvar

class B:
    def method(cls):
        pass
    method = classmethod(method)

In [41]:
c = A()
c.method(c)

c.myvar

1

## More examples of classmethods and staticmethos

In [53]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    @classmethod
    def from_birth_year(cls, birth_year):
        current_year = 2023
        age = current_year - birth_year
        return cls(name='Unknown', age = age)


p = Person('Arka', 31) #This is the generic initialization method where you supply all the necessary data

In [50]:
#However if you dont have the age data, but you have birth year and you want to create the instance of a class from that data, you can use the classmethod to do that like the following example
p = Person.from_birth_year(1992)

In [52]:
# p  is initialized with all the necessary data
p.name

'Unknown'

In [55]:
#example of staticmethod

class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    @classmethod
    def from_birth_year(cls, birth_year):
        current_year = 2023
        age = current_year - birth_year
        return cls(name='Unknown', age = age)
    
    @staticmethod
    def is_adult(age):
        return age>18
    

p = Person.from_birth_year(1992)
p.is_adult(32)  # Doesnot depend on the data of the instance

True

# Preserving Function Metadata 2hen writing decorators

In [56]:
import time
from functools import wraps


def timethis(func):
    '''
    Decorator that reports the execution time.
    '''

    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'{func.__name__} executed in {end - start} seconds')
        return result

    return wrapper


@timethis
def countdown(n):
    '''
    Counts down from n to 0
    '''
    while n>1:
        n -= 1
        

In [59]:
countdown(10)
countdown.__name__
countdown.__doc__

countdown executed in 3.814697265625e-06 seconds


'\n    Counts down from n to 0\n    '

## Unwrapping the decorator

In [65]:
countdown.__wrapped__(10)
 

## Defining a decorator that takes arguments

In [67]:
from functools import wraps
import logging


def logged(level, name=None, message=None):
    '''
    Add logging to a function. Level is the logging level,
    name is the logger name, and message is the log message.
    If name and message arent specified, They default to the function's
    module and name.
    '''
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate


#Example usecase
@logged(logging.DEBUG)
def add(x,y):
    return x+y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

In [71]:
spam()

spam


Spam!
